In [3]:
import numpy as np
import pandas as pd

In [7]:
df = pd.read_csv('diabetes.csv')

In [8]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [9]:
X = df.iloc[:, :-1].values
y = df.iloc[:,-1].values

In [10]:
from sklearn.preprocessing import StandardScaler

In [11]:
scaler = StandardScaler()

In [12]:
X = scaler.fit_transform(X)

In [13]:
X.shape

(768, 8)

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [120]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Input, Dropout

In [16]:
model = Sequential()
model.add(Input(shape=(8,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
history = model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.4486 - loss: 0.7176 - val_accuracy: 0.5714 - val_loss: 0.6610
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6375 - loss: 0.6404 - val_accuracy: 0.7468 - val_loss: 0.6116
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7076 - loss: 0.6024 - val_accuracy: 0.7727 - val_loss: 0.5811
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6996 - loss: 0.5806 - val_accuracy: 0.7727 - val_loss: 0.5565
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7366 - loss: 0.5612 - val_accuracy: 0.7727 - val_loss: 0.5356
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7579 - loss: 0.5236 - val_accuracy: 0.7727 - val_loss: 0.5186
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7563 - loss: 0.5271 - val_accuracy: 0.7727 - val_loss: 0.5075
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7461 - loss: 0.5215 - val_accuracy: 0.7792 - 

In [22]:
import keras_tuner as kt

**How to choose optimizer**

In [24]:
def build_model(hp):
  model = Sequential()
  model.add(Input(shape=(8,)))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  optimizer = hp.Choice('optimizer', values = ['SGD','RMSprop','Adam','Adadelta','Adagrad','Adafactor','Nadam'])
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model

In [25]:
tunner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5)

In [26]:
tunner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.5389610528945923

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 18s


In [34]:
tunner.search_space_summary()

Search space summary
Default search space size: 1
optimizer (Choice)
{'default': 'SGD', 'conditions': [], 'values': ['SGD', 'RMSprop', 'Adam', 'Adadelta', 'Adagrad', 'Adafactor', 'Nadam'], 'ordered': False}


In [41]:
models = tunner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [42]:
models.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [39]:
tunner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
optimizer: Adam
Score: 0.7727272510528564

Trial 1 summary
Hyperparameters:
optimizer: SGD
Score: 0.7597402334213257

Trial 0 summary
Hyperparameters:
optimizer: RMSprop
Score: 0.7467532753944397

Trial 3 summary
Hyperparameters:
optimizer: Adadelta
Score: 0.5844155550003052

Trial 4 summary
Hyperparameters:
optimizer: Adafactor
Score: 0.5389610528945923


In [40]:
tunner.get_best_hyperparameters()[0].values

{'optimizer': 'Adam'}

In [43]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7669 - loss: 0.4479 - val_accuracy: 0.7922 - val_loss: 0.4670
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8157 - loss: 0.3917 - val_accuracy: 0.7987 - val_loss: 0.4677
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7850 - loss: 0.4397 - val_accuracy: 0.7987 - val_loss: 0.4678
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7646 - loss: 0.4392 - val_accuracy: 0.7922 - val_loss: 0.4681
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8185 - loss: 0.3950 - val_accuracy: 0.7857 - val_loss: 0.4686
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7988 - loss: 0.4007 - val_accuracy: 0.7857 - val_loss: 0.4685
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7759 - loss: 0.4170 - val_accuracy: 0.7857 - val_loss: 0.4684
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7904 - loss: 0.4207 - val_accuracy: 0.79

In [63]:
def build_model(hp):
  model = Sequential()
  model.add(Input(shape=(8,)))
  units = hp.Int('units', min_value=8, max_value=128)
  model.add(Dense(units=units, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [65]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='mydir',project_name='node')

In [66]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7597402334213257

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 17s


In [67]:
tuner.results_summary()

Results summary
Results in mydir/node
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
units: 60
Score: 0.798701286315918

Trial 2 summary
Hyperparameters:
units: 105
Score: 0.798701286315918

Trial 1 summary
Hyperparameters:
units: 89
Score: 0.7857142686843872

Trial 4 summary
Hyperparameters:
units: 16
Score: 0.7597402334213257

Trial 3 summary
Hyperparameters:
units: 14
Score: 0.7467532753944397


In [68]:
tuner.get_best_hyperparameters()[0].values

{'units': 60}

In [69]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [70]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 60)                  │             540 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              61 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 601 (2.35 KB)

 Trainable params: 601 (2.35 KB)

 Non-trainable params: 0 (0.00 B)

In [71]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6,validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.7595 - loss: 0.5294 - val_accuracy: 0.8052 - val_loss: 0.5131
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7465 - loss: 0.5126 - val_accuracy: 0.7792 - val_loss: 0.4925
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7859 - loss: 0.4666 - val_accuracy: 0.7727 - val_loss: 0.4829
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7537 - loss: 0.4797 - val_accuracy: 0.7727 - val_loss: 0.4740
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7666 - loss: 0.4801 - val_accuracy: 0.7662 - val_loss: 0.4683
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7825 - loss: 0.4576 - val_accuracy: 0.7727 - val_loss: 0.4660
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7623 - loss: 0.4759 - val_accuracy: 0.7857 - val_loss: 0.4632
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7860 - loss: 0.4380 - val_accuracy: 0.78

In [72]:
loss, accuracy = model.evaluate(X_test, y_test)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8009 - loss: 0.4280 


In [73]:
print(accuracy)

0.8116883039474487


**How to select number of layers**

In [74]:
def build_model(hp):
  model = Sequential()
  model.add(Input(shape=(8,)))
  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    model.add(Dense(60, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [75]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='mydir',project_name='layers')

In [76]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 27s


In [77]:
tuner.results_summary()

Results summary
Results in mydir/layers
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
num_layers: 1
Score: 0.8181818127632141

Trial 3 summary
Hyperparameters:
num_layers: 8
Score: 0.8051947951316833

Trial 0 summary
Hyperparameters:
num_layers: 3
Score: 0.798701286315918

Trial 1 summary
Hyperparameters:
num_layers: 5
Score: 0.798701286315918

Trial 4 summary
Hyperparameters:
num_layers: 2
Score: 0.7922077775001526


In [78]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1}

In [79]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [80]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 60)                  │             540 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              61 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 601 (2.35 KB)

 Trainable params: 601 (2.35 KB)

 Non-trainable params: 0 (0.00 B)

In [81]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6,validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.7675 - loss: 0.5266 - val_accuracy: 0.8052 - val_loss: 0.5053
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7829 - loss: 0.4936 - val_accuracy: 0.7922 - val_loss: 0.4888
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7708 - loss: 0.4974 - val_accuracy: 0.7987 - val_loss: 0.4801
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7842 - loss: 0.4762 - val_accuracy: 0.7987 - val_loss: 0.4732
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7793 - loss: 0.4730 - val_accuracy: 0.7987 - val_loss: 0.4694
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7768 - loss: 0.4771 - val_accuracy: 0.8052 - val_loss: 0.4676
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7678 - loss: 0.4653 - val_accuracy: 0.8052 - val_loss: 0.4651
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7963 - loss: 0.4535 - val_accuracy: 0.79

In [82]:
loss, accuracy = model.evaluate(X_test, y_test)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8161 - loss: 0.4379


In [83]:
print(accuracy)

0.8246753215789795


In [121]:
def build_model(hp):
  model = Sequential()
  model.add(Input(shape=(8,)))
  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    model.add(
        Dense(
            units=hp.Int(f'units_{i}', min_value=8, max_value=128, step=8),
            activation=hp.Choice(f'activation_{i}', values=['relu', 'tanh','sigmoid'])))
    model.add(Dropout(hp.Choice(f'dropout_{i}', values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer', values = ['SGD','RMSprop','Adam','Adadelta','Adagrad','Adafactor','Nadam']),loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [122]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='mydir',project_name='final')

In [123]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 08s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 36s


In [124]:
tuner.results_summary()

Results summary
Results in mydir/final
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
num_layers: 1
units_0: 104
activation_0: relu
dropout_0: 0.1
optimizer: Nadam
Score: 0.7792207598686218

Trial 1 summary
Hyperparameters:
num_layers: 5
units_0: 128
activation_0: relu
dropout_0: 0.4
optimizer: Nadam
units_1: 8
activation_1: relu
dropout_1: 0.1
units_2: 8
activation_2: relu
dropout_2: 0.1
units_3: 8
activation_3: relu
dropout_3: 0.1
units_4: 8
activation_4: relu
dropout_4: 0.1
Score: 0.7142857313156128

Trial 2 summary
Hyperparameters:
num_layers: 4
units_0: 96
activation_0: sigmoid
dropout_0: 0.6
optimizer: Adafactor
units_1: 64
activation_1: relu
dropout_1: 0.5
units_2: 56
activation_2: tanh
dropout_2: 0.5
units_3: 56
activation_3: tanh
dropout_3: 0.1
units_4: 64
activation_4: sigmoid
dropout_4: 0.4
Score: 0.6428571343421936

Trial 4 summary
Hyperparameters:
num_layers: 10
units_0: 72
activation_0: sigmoid
dropout_0: 0.3
optim

In [125]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units_0': 104,
 'activation_0': 'relu',
 'dropout_0': 0.1,
 'optimizer': 'Nadam'}

In [126]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 2 variables whereas the saved optimizer has 11 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [127]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 104)                 │             936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 104)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             105 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,041 (4.07 KB)

 Trainable params: 1,041 (4.07 KB)

 Non-trainable params: 0 (0.00 B)

In [131]:
model.fit(X_train, y_train, batch_size=32, epochs=200, initial_epoch=6,validation_data=(X_test, y_test))

Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8423 - loss: 0.3826 - val_accuracy: 0.8117 - val_loss: 0.4606
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8365 - loss: 0.3841 - val_accuracy: 0.8182 - val_loss: 0.4609
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8081 - loss: 0.4132 - val_accuracy: 0.8117 - val_loss: 0.4603
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7941 - loss: 0.3990 - val_accuracy: 0.8182 - val_loss: 0.4577
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8087 - loss: 0.3877 - val_accuracy: 0.8117 - val_loss: 0.4601
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8001 - loss: 0.4356 - val_accuracy: 0.8182 - val_loss: 0.4581
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8320 - loss: 0.3787 - val_accuracy: 0.8182 - val_loss: 0.4582
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7984 - loss: 0.4188 - val_accuracy: 0.818

In [132]:
loss, accuracy = model.evaluate(X_test, y_test)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7801 - loss: 0.4641 


In [133]:
print(accuracy)

0.798701286315918
